In [1]:
import csv
import os
from collections import Counter
from collections import defaultdict
from string import punctuation

import enchant
import pandas as pd
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.util import ngrams

# Lexicon

In [2]:
def remove_accent_marks(w):
    r = w
    r = r.replace('á', 'a')
    r = r.replace('é', 'e')
    r = r.replace('í', 'i')
    r = r.replace('ó', 'o')
    r = r.replace('ú', 'u')
    return r

In [3]:
%%time

lexicon = {}
with open('lexicon/es_lexicon.csv') as f:
    reader = csv.reader(
        f,
        delimiter=' ',
    )
    for row in reader:
        w = remove_accent_marks(row[0])
        lexicon[w] = []
        for i in range(1, len(row[1:]), 2):
            entry = {}
            entry['lemma'] = remove_accent_marks(row[i].lower())
            entry['eagle'] = remove_accent_marks(row[i+1].lower())
            lexicon[w].append(entry)

CPU times: user 4.13 s, sys: 127 ms, total: 4.25 s
Wall time: 4.22 s


In [4]:
len(lexicon)

500949

In [5]:
def is_vowel(c):
    return c in 'aeiouAEIOUáéíóúÁÉÍÓÚüÜ'

def next_level(w):
    result = [w]
    if is_vowel(w[0]):
        result.append('h' + w)
    elif w.startswith('h'):
        result.append(w[1:])
    if "q'" in w:
        result.append(w.replace("q'", 'que'))
    for i in range(len(w)):
        if w[i] == 'v':
            result.append(w[:i] + 'b' + w[i+1:])
        elif w[i] == 'b':
            result.append(w[:i] + 'v' + w[i+1:])
            result.append(w[:i] + 'd' + w[i+1:])
        elif w[i] == 'd':
            result.append(w[:i] + 'b' + w[i+1:])
        elif w[i] == 'c' and i < len(w) - 1 and w[i+1] in 'ei':
            result.append(w[:i] + 's' + w[i+1:])
        elif w[i] == 's' and i < len(w) - 1 and w[i+1] in 'ei':
            result.append(w[:i] + 'c' + w[i+1:])
    return result

def combinations_aux(visited, result):
    while visited:
        w = visited[0]
        visited = visited[1:]
        if w not in result:
            result.append(w)
            combs = next_level(w)
            visited.extend(combs)        
    return result

def combinations(w):
    return combinations_aux([w], [])

In [6]:
es = enchant.Dict('es_ES')

# Analysis

In [7]:
excel = pd.ExcelFile('ColombiaWork2.xlsx')

In [8]:
df1 = excel.parse(0)
df1 = df1.fillna('')

In [9]:
df2 = excel.parse(1)
df2 = df2.fillna('')

In [10]:
df3 = excel.parse(2)
df3 = df3.fillna('')

In [11]:
df4 = excel.parse(3)
df4 = df4.fillna('')

In [12]:
df5 = excel.parse(4)
df5 = df5.fillna('')

In [13]:
dfs = [df1, df2, df3, df4, df5]

In [14]:
discard_columns = ['Filename', 'Date (YYYY-MM-DD)', 'Name', 'Gender', 'Age', 'City', 'Drawing', 'Title', 'Story']

In [15]:
for i, df in enumerate(dfs):
    for d in discard_columns:
        if d not in df.columns:
            print(i + 1, d)

In [16]:
df1.drop(discard_columns, inplace=True, axis=1)

In [17]:
df2.drop(discard_columns, inplace=True, axis=1)

In [18]:
df3.drop(discard_columns, inplace=True, axis=1)

In [19]:
df4.drop(discard_columns, inplace=True, axis=1)

In [20]:
df5.drop(discard_columns, inplace=True, axis=1)

In [21]:
dfs = [df1, df2, df3, df4, df5]

## Palabras definidas

In [22]:
palabras_definidas = set()

for df in dfs:
    for col in df.columns:
        if not (col.startswith('Question') or col.startswith('Answer')):
            palabras_definidas.add(col)

In [23]:
len(palabras_definidas)

166

In [24]:
palabras_definidas

{'Abandono',
 'Abrazar',
 'Abuelos',
 'Acompañar',
 'Agradecimiento',
 'Agua',
 'Alegria',
 'Alivio',
 'Amar',
 'Amistad',
 'Amor',
 'Aprender',
 'Arma',
 'Armonía',
 'Asesinato',
 'Astucia',
 'Ayudar',
 'Belleza',
 'Besar',
 'Blanco',
 'Bomba',
 'Bondad',
 'Brincar',
 'Cambio',
 'Cariño',
 'Carta',
 'Casa',
 'Castigar',
 'Colombia',
 'Comida',
 'Compartir',
 'Competencia',
 'Competencia.1',
 'Confianza',
 'Conocimiento',
 'Crecer',
 'Cuento',
 'Cuidar',
 'Desear',
 'Desperdiciar',
 'Desplazamiento',
 'Despreciar',
 'Dolor',
 'Engañar',
 'Enseñanza',
 'Esconder',
 'Escribir',
 'Escuela',
 'Espantoso',
 'Esperanza',
 'Estudiantes',
 'Familia',
 'Feura',
 'Fraternidad',
 'Frio',
 'Fuego',
 'Fuera',
 'Futuro',
 'Golpear',
 'Guerra',
 'Guerrilla',
 'Herir',
 'Hermandad',
 'Hijos',
 'Hipocresía',
 'Historia',
 'Hombre',
 'Humildad',
 'Ignorancia',
 'Injusticia',
 'Invisible',
 'Juego',
 'Justicia',
 'Lastimar',
 'Leer',
 'Limpieza',
 'Llorar',
 'Luz',
 'Mafioso',
 'Masacre',
 'Matar',
 'Mej

## Words

In [25]:
punctuation = punctuation + '¡¿“”…'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~¡¿“”…'

In [26]:
def normalize(text):
    text = text.lower()
    text = remove_accent_marks(text)
    for p in punctuation:
        text = text.replace(p, '')
    return text

In [27]:
total_words = []

for df in dfs:
    for c in df:
        for x in df[c]:
            text = normalize(x)
            words = [w for w in word_tokenize(text)]
            total_words.extend(words)

In [28]:
len(total_words)

23063

In [29]:
total_words_freq = Counter(total_words)

In [30]:
total_words_freq.most_common(10)

[('que', 1731),
 ('la', 1064),
 ('y', 771),
 ('de', 697),
 ('a', 486),
 ('paz', 461),
 ('es', 422),
 ('se', 418),
 ('en', 414),
 ('con', 385)]

In [31]:
stop_words = [remove_accent_marks(w) for w in stopwords.words('spanish')]

In [32]:
total_nonstop_words = [w for w in total_words if w not in stop_words]

In [33]:
len(total_nonstop_words)

10593

In [34]:
total_nonstop_words_freq = Counter(total_nonstop_words)

In [35]:
total_nonstop_words_freq.most_common(10)

[('paz', 461),
 ('guerra', 343),
 ('da', 245),
 ('alimenta', 175),
 ('cosas', 161),
 ('personas', 154),
 ('persona', 132),
 ('pudieras', 131),
 ('dirias', 131),
 ('5', 127)]

## Spelling

In [36]:
%%time

correct_spelling = {}

for w in total_words_freq:
    if w not in correct_spelling:
        if w in lexicon:
            correct_spelling[w] = w
        else:
            combs = combinations(w)
            freqs = [(c, total_words_freq[c]) for c in combs if c in total_words_freq and c in lexicon]
            if freqs:
                freqs.sort(key=lambda x: x[1], reverse=True)
                correct_spelling[w] = freqs[0][0]
            else:
                freqs = [(c, total_words_freq[c]) for c in es.suggest(w) if c in total_words_freq]
                if freqs:
                    freqs.sort(key=lambda x: x[1], reverse=True)
                    correct_spelling[w] = freqs[0][0]
                else:
                    correct_spelling[w] = w

CPU times: user 13.6 s, sys: 6.56 ms, total: 13.6 s
Wall time: 13.6 s


In [37]:
len(correct_spelling) # it includes stop words

2802

In [38]:
'y' in stop_words

True

In [39]:
total_corrected_nonstop_words = [correct_spelling[w] for w in total_nonstop_words]
total_corrected_nonstop_words = [w for w in total_corrected_nonstop_words if w not in stop_words]

In [40]:
len(total_corrected_nonstop_words)

10078

In [41]:
total_corrected_nonstop_words_freq = Counter(total_corrected_nonstop_words)

In [42]:
total_corrected_nonstop_words_freq.most_common(200)

[('paz', 468),
 ('guerra', 423),
 ('da', 245),
 ('alimenta', 175),
 ('cosas', 161),
 ('personas', 155),
 ('hablar', 141),
 ('persona', 135),
 ('pudieras', 133),
 ('dirias', 131),
 ('pais', 113),
 ('vida', 110),
 ('miedo', 92),
 ('amor', 89),
 ('sueña', 88),
 ('q', 87),
 ('hacer', 83),
 ('alegria', 82),
 ('ser', 82),
 ('violencia', 77),
 ('alguien', 68),
 ('dice', 65),
 ('años', 64),
 ('habla', 64),
 ('tener', 63),
 ('hace', 62),
 ('diria', 61),
 ('viste', 50),
 ('mundo', 48),
 ('familia', 46),
 ('tristeza', 44),
 ('entrar', 43),
 ('bien', 43),
 ('gente', 42),
 ('demas', 42),
 ('vez', 40),
 ('quiero', 38),
 ('mal', 37),
 ('muerte', 37),
 ('15', 35),
 ('razones', 35),
 ('tranquilidad', 34),
 ('siento', 33),
 ('odio', 33),
 ('matar', 33),
 ('etc', 33),
 ('significa', 31),
 ('sueñas', 30),
 ('gracias', 30),
 ('maldad', 29),
 ('bueno', 28),
 ('solo', 28),
 ('armonia', 28),
 ('daño', 27),
 ('hacen', 27),
 ('felicidad', 27),
 ('asi', 26),
 ('mejor', 26),
 ('preguntaria', 24),
 ('niños', 24),


## Topics

In [43]:
topics = {
    'paz': [('paz', 468),],
    'ámbito_temporal': [('años', 64), ('siempre', 21), ('nunca', 16), ('dia', 12),],
    'familia': [('familia', 46), ('mama', 22), ('familias', 20), ('padres', 12), ('papa', 11), ('hermano', 10),],
    'vida': [('vida', 110), ('vivir', 21), ('vidas', 9),],
    'gente': [('personas', 155), ('persona', 135), ('alguien', 68), ('demas', 42), ('gente', 42), ('niños', 24),],
    'amor': [('amor', 89), ('cariño', 20), ('corazon', 15), ('querer', 13), ('amar', 13), ('ama', 8),],
    'niños': [],
    'perdón': [('perdona', 13), ('perdon', 12), ('perdonar', 9),],
    'ámbito_espacial': [('pais', 113), ('mundo', 48), ('lugar', 13), ('habana', 9), ('calle', 8), ('cuba', 8), ('planeta', 8),],
    'amistad': [('amigos', 19), ('amistad', 8),],
    'tolerancia': [('respeto', 23), ('respetar', 14),],
    'patriotismo': [],
    'violencia': [('guerra', 423), ('violencia', 77), ('muerte', 37), ('matar', 33), ('daño', 27), ('pelear', 21), ('dolor', 15), ('sufrimiento', 14), ('destruir', 14), ('armas', 11), ('muertes', 8),],
    'compañeros': [],
    'solidaridad': [('ayudar', 23), ('ayuda', 12), ('necesita', 8),],
    'comunicación': [('hablar', 141), ('dirias', 131), ('dice', 65), ('habla', 64), ('diria', 61), ('razones', 35), ('hablando', 13), ('decir', 12), ('palabra', 8), ('razon', 8),],
    'conflicto': [('problemas', 18),],
    'solución': [],
    'felicidad': [('alegria', 82), ('felicidad', 27), ('feliz', 10), ('alegre', 9),],
    'religión': [('dios', 15), ('fe', 8),],
    'animales': [],
    'educación': [],
    'rencor': [('odio', 33), ('rencor', 13),],
    'agradecimiento': [('gracias', 30), ],
    'jóvenes': [],
    'paz_interior': [('tranquilidad', 34), ('armonia', 28), ('tranquilo', 13),],
    'agua': [],
    'tristeza': [('tristeza', 44), ('triste', 9),],
    'miedo': [('miedo', 92), ('oscuridad', 15), ('temor', 14),],
    'deseo': [('pudieras', 133), ('sueña', 88), ('quiero', 38), ('sueñas', 30), ('sueño', 24), ('pudiera', 21), ('futuro', 11), ('seguir', 11), ('adelante', 9), ('cumplir', 9), ('deseo', 9), ('sueños', 8),],
    'ética': [('bien', 43), ('mal', 37), ('maldad', 29), ('bueno', 28), ('malo', 21), ('malos', 13),],
    'ropa': [('camisa', 24), ('pantalon', 22), ('zapatos', 11),],
    'dinero': [('plata', 17),],
    'sentimiento': [('sentimiento', 15), ('sentir', 11),],
    'color': [('blanco', 17), ('verde', 14), ('blanca', 14), ('negro', 9), ('azul', 8)],
    'justicia': [('justicia', 14), ('injusticia', 13),],
    'libertad': [('libertad', 13),],
    'inocencia': [('inocentes', 10), ('inocente', 10),],
    'abandono': [('abandono', 10),],
    'autoridad': [('presidente', 8),],
}

In [44]:
tups = [(k, sum(map(lambda x: x[1], words)), words) for k, words in topics.items()]
tups.sort(key=lambda x: x[1], reverse=True)

In [45]:
tups

[('violencia',
  680,
  [('guerra', 423),
   ('violencia', 77),
   ('muerte', 37),
   ('matar', 33),
   ('daño', 27),
   ('pelear', 21),
   ('dolor', 15),
   ('sufrimiento', 14),
   ('destruir', 14),
   ('armas', 11),
   ('muertes', 8)]),
 ('comunicación',
  538,
  [('hablar', 141),
   ('dirias', 131),
   ('dice', 65),
   ('habla', 64),
   ('diria', 61),
   ('razones', 35),
   ('hablando', 13),
   ('decir', 12),
   ('palabra', 8),
   ('razon', 8)]),
 ('paz', 468, [('paz', 468)]),
 ('gente',
  466,
  [('personas', 155),
   ('persona', 135),
   ('alguien', 68),
   ('demas', 42),
   ('gente', 42),
   ('niños', 24)]),
 ('deseo',
  391,
  [('pudieras', 133),
   ('sueña', 88),
   ('quiero', 38),
   ('sueñas', 30),
   ('sueño', 24),
   ('pudiera', 21),
   ('futuro', 11),
   ('seguir', 11),
   ('adelante', 9),
   ('cumplir', 9),
   ('deseo', 9),
   ('sueños', 8)]),
 ('ámbito_espacial',
  207,
  [('pais', 113),
   ('mundo', 48),
   ('lugar', 13),
   ('habana', 9),
   ('calle', 8),
   ('cuba', 8